### Initialize PySpark et Charger les Données

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("NYC_Taxi_EDA") \
    .config("spark.sql.adaptive.enabled","true") \
    .getOrCreate()
    
df = spark.read.parquet("../data/dataset.parquet")


### Overview du Dataset

In [32]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [33]:
print(f"Nombre de lignes : {df.count()}")
print(f"Nombre de colonnes : {len(df.columns)}")

Nombre de lignes : 3475226
Nombre de colonnes : 20


In [34]:
print(f"schema du dataset : {df.printSchema()}")

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

schema du dataset : None


In [35]:
df.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+------------------+--------------------+-------------------+-------------------+
|summary|          VendorID|   passenger_count|    trip_distance|       RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+-------

In [36]:
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double'),
 ('cbd_congestion_fee', 'double')]

####  Détecter les Valeurs Manquantes et Zéros Anormaux

In [37]:
missing_counts = df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
])

print(missing_counts)

DataFrame[VendorID: bigint, tpep_pickup_datetime: bigint, tpep_dropoff_datetime: bigint, passenger_count: bigint, trip_distance: bigint, RatecodeID: bigint, store_and_fwd_flag: bigint, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: bigint, extra: bigint, mta_tax: bigint, tip_amount: bigint, tolls_amount: bigint, improvement_surcharge: bigint, total_amount: bigint, congestion_surcharge: bigint, Airport_fee: bigint, cbd_congestion_fee: bigint]


##### POURCENTAGE DE VALEURS MANQUANTES: 

In [38]:
total_rows = df.count()
print("POURCENTAGE DE VALEURS MANQUANTES:")
for col_name in df.columns:
    missing_count = df.filter(col(col_name).isNull()).count()
    percentage = (missing_count / total_rows) * 100
    if percentage > 0:
        print(f"{col_name}: {percentage:.2f}%")

POURCENTAGE DE VALEURS MANQUANTES:
passenger_count: 15.54%
RatecodeID: 15.54%
store_and_fwd_flag: 15.54%
congestion_surcharge: 15.54%
Airport_fee: 15.54%


##### Zeros anormaux

In [40]:
zero_checks = [
    "passenger_count",
    "trip_distance", 
    "fare_amount",
    "total_amount",
    "RatecodeID"
]

for col_name in zero_checks:
    zero_count = df.filter(col(col_name) == 0).count()
    print(f"{col_name}: {zero_count} valeurs à zéro ({(zero_count/total_rows)*100:.2f}%)")

passenger_count: 24656 valeurs à zéro (0.71%)
trip_distance: 90893 valeurs à zéro (2.62%)
fare_amount: 1398 valeurs à zéro (0.04%)
total_amount: 559 valeurs à zéro (0.02%)
RatecodeID: 0 valeurs à zéro (0.00%)


### DÉTECTION DES DUPLICATS:

In [41]:
distinct_rows = df.distinct().count()
duplicates = total_rows - distinct_rows
print(f"   Lignes totales: {total_rows:,}")
print(f"   Lignes uniques: {distinct_rows:,}")
print(f"   Duplicats: {duplicates:,} ({(duplicates/total_rows)*100:.2f}%)")

   Lignes totales: 3,475,226
   Lignes uniques: 3,475,226
   Duplicats: 0 (0.00%)


### sauvgarder bronze df

In [43]:
df.write \
  .mode("overwrite") \
  .parquet("../data/bronze/nyc_taxi/")